In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt

In [2]:
sen_2018 = pd.read_csv('senate_polls_2018_inprogress.csv')

In [4]:
#Move from one row per candidate to one row per poll
#What's the last poll we've been working on?
oldno = -10
#What's the index of that last poll?
masterindex = 0
for index, pollno in enumerate(sen_2018['pollno']):
    #If we've seen this poll_id before, this row deals with a second, third, etc candidate:
    if oldno == pollno:
        #If we haven't set stats for candidate 2 yet:
        if np.isnan(sen_2018['cand2_pct'][masterindex]):
            #Set them equal to this row
            sen_2018['cand2_pct'][masterindex] = sen_2018['cand1_pct'][index]
            sen_2018['cand2_name'][masterindex] = sen_2018['cand1_name'][index]
        #If we're still on this poll, there could be a third candidate, but their name isn't tracked
        elif np.isnan(sen_2018['cand3_pct'][masterindex]):
            sen_2018['cand3_pct'][masterindex] = sen_2018['cand1_pct'][index]
        #Some polls track even more candidates, but our model doesn't, so just don't do anything 
        #if there's even more rows from this poll
    else:
        #This is a new poll, so update the reference information and keep going
        oldno = pollno
        masterindex = index

C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\marcu\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\marc

In [5]:
#Remove all superfluous rows
sen_2018 = sen_2018[~np.isnan(sen_2018['cand2_pct'])]

In [6]:
sen_2018['margin_poll'] = (sen_2018['cand1_pct'] - sen_2018['cand2_pct'])

In [7]:
#Credit to Mike Shultz for creating this
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'}

In [8]:
state_key = {state: abbr for abbr, state in states.items()}

In [10]:
sen_results = pd.read_csv('senate_results_2018.csv')

In [11]:
sen_results = sen_results.set_index('State').transpose()

In [12]:
#Add in results data and data analysis
#Still need to rework the rightcall code
sen_2018['location'] = sen_2018['state']
sen_2018 = sen_2018.reset_index()
for index in range(len(sen_2018)):
    state = sen_2018['state'][index]
    sen_2018['location'][index] = state_key[state]
    abbr = sen_2018['location'][index]
    sen_2018['cand1_actual'][index] = sen_results[abbr][0]
    sen_2018['cand2_actual'][index] = sen_results[abbr][1]
    sen_2018['margin_actual'] = sen_2018['cand1_actual'] - sen_2018['cand2_actual']
    sen_2018['margin_poll'] = sen_2018['cand1_pct'] - sen_2018['cand2_pct']
    sen_2018['bias'] = sen_2018['margin_poll'] - sen_2018['margin_actual']
    sen_2018['error'] = abs(sen_2018['bias'])
    #if ((sen_2018['cand1_pct'] > sen_2018['cand2_pct']) and (sen_2018['cand1_actual'] > sen_2018['cand2_actual'])) or ((sen_2018['cand1_pct'] < sen_2018['cand2_pct']) and (sen_2018['cand1_actual'] < sen_2018['cand2_actual'])):
        #sen_2018['rightcall'] = 1
    #else:
         #sen_2018['rightcall'] = 0

C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [14]:
sen_2018

,index,pollno,race,year,Location,state,type_simple,pollster,sample_size,population_full,...,cand3_pct,margin_poll,election_date,cand1_actual,cand2_actual,margin_actual,error,bias,rightcall,location
0,0,56923,2018_Sen_,2018,NaN,Mississippi,Sen-G,Change Research,1211.0,lv,...,NaN,-5.0,11/27/2018,39.47,58.49,-19.02,14.02,14.02,NaN,MS
1,2,56921,2018_Sen_,2018,NaN,Mississippi,Sen-G,JMC Analytics/Bold Blue Campaigns,684.0,lv,...,NaN,-10.0,11/27/2018,39.47,58.49,-19.02,9.02,9.02,NaN,MS
2,4,56815,2018_Sen_,2018,NaN,Michigan,Sen-G,Mitchell Research & Communications,827.0,lv,...,NaN,3.0,11/6/2018,52.26,35.76,16.50,13.50,-13.50,NaN,MI
3,6,56825,2018_Sen_,2018,NaN,Florida,Sen-G,Trafalgar Group,1484.0,lv,...,NaN,-1.7,11/6/2018,49.94,50.06,-0.12,1.58,-1.58,NaN,FL
4,8,56827,2018_Sen_,2018,NaN,Arizona,Sen-G,Trafalgar Group,1217.0,lv,...,NaN,-2.1,11/6/2018,50.00,47.60,2.40,4.50,-4.50,NaN,AZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,1937,53556,2018_Sen_,2018,NaN,Missouri,Sen-G,Remington Research Group,928.0,lv,...,NaN,-6.0,11/6/2018,45.57,51.38,-5.81,0.19,-0.19,NaN,MO
597,1939,52661,2018_Sen_,2018,NaN,Nevada,Sen-G,Public Policy Polling,648.0,v,...,NaN,1.0,11/6/2018,50.41,45.38,5.03,4.03,-4.03,NaN,NV
598,1941,53946,2018_Sen_,2018,NaN,Massachusetts,Sen-G,MassINC Polling Group,504.0,rv,...,NaN,31.0,11/6/2018,60.34,36.17,24.17,6.83,6.83,NaN,MA
599,1943,52630,2018_Sen_,2018,NaN,Texas,Sen-G,Texas Lyceum,1000.0,a,...,NaN,0.0,11/6/2018,48.33,50.89,-2.56,2.56,2.56,NaN,TX


In [15]:
senTest = sen_2018.groupby('methodology').mean()

In [17]:
senTest['error']

methodology
Automated Phone      5.086739
IVR/Live Phone       3.992000
IVR/Online           4.504286
Live Phone           4.970757
Live Phone/Online    2.180833
Online               5.101550
Name: error, dtype: float64